In [50]:
import ifcopenshell
import ifcopenshell.geom
import pyvista as pv
import numpy as np
import trimesh
from scipy.spatial import KDTree
from scipy.spatial import Delaunay

import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

In [2]:
def visualize_geometry(geometry_dict):
    # Create a PyVista plotter
    plotter = pv.Plotter()
    
    # Iterate through the dictionary
    for key, geometries in geometry_dict.items():
        # Parse the key to get color and style
        color, style = key.split('-')
        
        # Set up the color
        rgb_color = pv.Color(color).float_rgb
        
        # Set up the style
        opacity = 1.0
        if style == 'wireframe':
            style = 'wireframe'
        elif style == 'solid':
            style = 'surface'
        elif style == 'transparent':
            style = 'surface'
            opacity = 0.5
        else:
            raise ValueError(f"Unknown style: {style}")
        
        # Add each geometry to the plotter
        for geometry in geometries:
            if isinstance(geometry, np.ndarray):
                # It's a point cloud
                point_cloud = pv.PolyData(geometry)
                plotter.add_points(point_cloud, color=rgb_color, opacity=opacity, point_size=5)
            else:
                # It's a mesh
                if not isinstance(geometry, pv.DataSet):
                    geometry = pv.wrap(geometry)
                
                if style == 'wireframe':
                    plotter.add_mesh(geometry.extract_all_edges(), color=rgb_color, opacity=opacity, line_width=2)
                else:
                    plotter.add_mesh(geometry, color=rgb_color, opacity=opacity, style=style)
    
    # Show the plot
    plotter.show()

In [3]:
def process_ifc_file(ifc_file_path):
    # Load the IFC file
    ifc_file = ifcopenshell.open(ifc_file_path)
    
    # Set up the geometry settings
    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, True)
    
    # Initialize the dictionary to store meshes
    mesh_dict = {}
    
    # List of IFC types to process 
    ifc_types = [
        "IfcWall", "IfcSlab", "IfcBeam", "IfcColumn", "IfcDoor", "IfcWindow",
        "IfcRoof", "IfcStair", "IfcRailing", "IfcFurnishingElement"
    ]
    
    # Process each IFC type
    for ifc_type in ifc_types:
        elements = ifc_file.by_type(ifc_type)
        meshes = []
        
        for element in elements:
            try:
                # Create geometry from the IFC element
                shape = ifcopenshell.geom.create_shape(settings, element)
                
                # Extract vertices and faces
                verts = shape.geometry.verts
                faces = shape.geometry.faces
                
                # Reshape vertices array
                vertices = np.array(verts).reshape((-1, 3))
                
                # Create faces array (add first vertex to each face for pyvista)
                faces_array = np.column_stack((np.full((len(faces) // 3, 1), 3), 
                                               np.array(faces).reshape((-1, 3))))
                
                # Create pyvista PolyData
                mesh = pv.PolyData(vertices, faces_array)
                
                meshes.append(mesh)
            except RuntimeError:
                # Skip elements that don't have geometry
                continue
        
        # Add non-empty mesh lists to the dictionary
        if meshes:
            mesh_dict[f"{ifc_type[3:].lower()}s"] = meshes
    
    return mesh_dict

In [4]:
# load up an example IFC shell 
ifc_dict = process_ifc_file("/Users/r2d2/Desktop/ariadne/test.ifc")

In [5]:
visualize_geometry({"red-wireframe":ifc_dict['slabs'], "green-solid":ifc_dict["walls"]})

Widget(value='<iframe src="http://localhost:56387/index.html?ui=P_0x12ee0dc10_0&reconnect=auto" class="pyvista…

In [24]:
print(ifc_dict["walls"][0])
visualize_geometry({"blue-wireframe":[ifc_dict["slabs"][0]]})

PolyData (0x12b1567a0)
  N Cells:    348
  N Points:   152
  N Strips:   0
  X Bounds:   0.000e+00, 1.830e+01
  Y Bounds:   1.370e+01, 1.400e+01
  Z Bounds:   -3.000e+00, -3.000e-01
  N Arrays:   0


Widget(value='<iframe src="http://localhost:56387/index.html?ui=P_0x399b5af00_6&reconnect=auto" class="pyvista…

In [25]:
def process_mesh(pv_mesh, up_vector=np.array([0, 0, 1]), angle_threshold=np.radians(45)):
    # Convert PyVista mesh to Trimesh
    vertices = pv_mesh.points
    faces = pv_mesh.faces.reshape(-1, 4)[:, 1:]
    tri_mesh = trimesh.Trimesh(vertices=vertices, faces=faces)

    # Calculate face normals
    face_normals = tri_mesh.face_normals

    # Calculate dot product with up vector
    dot_products = np.dot(face_normals, up_vector)

    # Filter faces based on orientation
    upward_mask = dot_products > np.cos(angle_threshold)

    # Function to create a PyVista mesh from a single face
    def face_to_pyvista(vertices, face):
        face_vertices = vertices[face]
        return pv.PolyData(face_vertices, np.array([[3, 0, 1, 2]]))

    # Create lists of individual face meshes
    upward_faces = []
    other_faces = []

    for i, face in enumerate(tri_mesh.faces):
        face_mesh = face_to_pyvista(tri_mesh.vertices, face)
        if upward_mask[i]:
            upward_faces.append(face_mesh)
        else:
            other_faces.append(face_mesh)

    return {
        "upward_faces": upward_faces,
        "other_faces": other_faces
    }

In [26]:
example_mesh = process_mesh(ifc_dict["slabs"][0])

In [27]:
visualize_geometry({"red-solid":example_mesh["other_faces"], "green-solid":example_mesh["upward_faces"], "blue-wireframe":[ifc_dict["walls"][0]]})

Widget(value='<iframe src="http://localhost:56387/index.html?ui=P_0x363123a10_7&reconnect=auto" class="pyvista…

In [20]:
def poisson_disk_sampling(meshes, radius, k=30, return_pv=True):
    def sample_single_mesh(mesh, radius):
        # Convert PyVista mesh to Trimesh if necessary
        if isinstance(mesh, pv.PolyData):
            tri_mesh = trimesh.Trimesh(vertices=mesh.points, faces=mesh.faces.reshape(-1, 4)[:, 1:])
        else:
            tri_mesh = mesh

        # Calculate an appropriate count based on mesh area and radius
        approx_count = int(tri_mesh.area / (np.pi * radius**2))
        count = max(approx_count, 10)  # Ensure at least 10 points

        # Perform Poisson disk sampling with warnings suppressed
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            samples, face_indices = trimesh.sample.sample_surface_even(tri_mesh, count=count, radius=radius)
        
        return samples, face_indices

    all_samples = []
    all_face_indices = []
    
    # Ensure meshes is a list
    if not isinstance(meshes, list):
        meshes = [meshes]
    
    for mesh in meshes:
        samples, face_indices = sample_single_mesh(mesh, radius)
        if len(samples) > 0:
            all_samples.append(samples)
            all_face_indices.append(face_indices)
    
    # Combine all samples
    combined_samples = np.vstack(all_samples) if all_samples else np.array([])
    
    if return_pv:
        # Convert to PyVista PolyData for visualization
        point_cloud = pv.PolyData(combined_samples)
        return point_cloud
    else:
        return combined_samples

In [28]:
points = poisson_disk_sampling(example_mesh["upward_faces"], 0.1)

In [29]:
visualize_geometry({"red-solid":example_mesh["other_faces"], "green-solid":example_mesh["upward_faces"], "blue-wireframe":[ifc_dict["walls"][0]], "blue-solid":[points]})

Widget(value='<iframe src="http://localhost:56387/index.html?ui=P_0x36f2193d0_8&reconnect=auto" class="pyvista…

In [13]:

def filter_points_near_meshes(points, meshes, distance_threshold):
    """
    Filter out points that are within a specified distance from any of the given meshes.
    
    Parameters:
    points (np.ndarray or pv.PolyData): Point cloud data
    meshes (list): List of PyVista mesh objects
    distance_threshold (float): Distance threshold for filtering
    
    Returns:
    pv.PolyData: Filtered point cloud
    """
    # Convert points to PyVista PolyData if it's a numpy array
    if isinstance(points, np.ndarray):
        points = pv.PolyData(points)
    
    # Combine all non-empty meshes into a single mesh
    combined_mesh = pv.PolyData()
    for mesh in meshes:
        if mesh.n_cells > 0:
            combined_mesh += mesh
    
    # Check if the combined mesh is empty
    if combined_mesh.n_cells == 0:
        print("Warning: All input meshes are empty. Returning original points.")
        return points
    
    # Use SciPy's KDTree to compute distances
    tree = KDTree(combined_mesh.points)
    distances, _ = tree.query(points.points, k=1)
    
    # Create a mask for points that are farther than the threshold
    mask = distances > distance_threshold
    
    # Apply the mask to filter the points
    filtered_points = points.extract_points(mask)
    
    return filtered_points

In [30]:
filtered_points = filter_points_near_meshes(points, example_mesh["other_faces"], 0.5)

In [31]:
visualize_geometry({"red-solid":example_mesh["other_faces"], "green-solid":example_mesh["upward_faces"], "blue-wireframe":[ifc_dict["walls"][0]], "blue-transparent":[points], "yellow-solid":[filtered_points]})

Widget(value='<iframe src="http://localhost:56387/index.html?ui=P_0x399b60860_9&reconnect=auto" class="pyvista…

In [39]:
horizontal_meshes = []
vertical_meshes = []

for key in ifc_dict.keys():
    # get the meshs associated with this key
    if key in ["slabs","stairs"]:
        meshes = ifc_dict[key]
        for mesh in meshes:
            processed_mesh = process_mesh(mesh)
            horizontal_meshes.extend(processed_mesh["upward_faces"])
    if key == "walls":
        meshes = ifc_dict[key]
        for mesh in meshes:
            processed_mesh = process_mesh(mesh)
            vertical_meshes.extend(processed_mesh["other_faces"])

In [40]:
# visualize_geometry({"green-solid": horizontal_meshes, "red-wireframe": vertical_meshes})

In [41]:
# create a point cloud on the horizontal meshes
point_cloud = poisson_disk_sampling(horizontal_meshes, 0.1)

only got 3/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 3/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 1/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 1/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 1/10 samples!
only got 3/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/10 samples!
only got 2/

In [44]:
visualize_geometry({"red-solid":[point_cloud], "green-wireframe":vertical_meshes})

Widget(value='<iframe src="http://localhost:56387/index.html?ui=P_0x39565cd70_13&reconnect=auto" class="pyvist…

In [48]:
filtered_points = filter_points_near_meshes(point_cloud, vertical_meshes, 0.5)

In [49]:
visualize_geometry({"green-solid":[filtered_points]})

Widget(value='<iframe src="http://localhost:56387/index.html?ui=P_0x3c7d79e50_16&reconnect=auto" class="pyvist…

In [55]:
def create_delaunay_meshes(points, horizontal_threshold, vertical_threshold):
    """
    Create 3D Delaunay meshes from a point cloud, respecting distance thresholds.
    
    Parameters:
    points (np.ndarray or pv.DataSet): Point coordinates or PyVista dataset
    horizontal_threshold (float): Maximum allowed horizontal distance between connected points
    vertical_threshold (float): Maximum allowed vertical distance between connected points
    
    Returns:
    list of pv.PolyData: List of PyVista mesh objects
    """
    # Convert input to numpy array if it's a PyVista object
    if isinstance(points, pv.DataSet):
        points = points.points
    
    # Ensure points is a 2D numpy array
    points = np.asarray(points)
    if points.ndim != 2 or points.shape[1] != 3:
        raise ValueError("Points must be a 2D array with shape (N, 3)")

    # Perform Delaunay triangulation
    tri = Delaunay(points)
    
    # Filter simplices based on distance thresholds
    valid_simplices = []
    for simplex in tri.simplices:
        simplex_points = points[simplex]
        
        # Check all edges of the tetrahedron
        edges = [
            (simplex_points[0], simplex_points[1]),
            (simplex_points[0], simplex_points[2]),
            (simplex_points[0], simplex_points[3]),
            (simplex_points[1], simplex_points[2]),
            (simplex_points[1], simplex_points[3]),
            (simplex_points[2], simplex_points[3])
        ]
        
        valid_edge_count = 0
        for p1, p2 in edges:
            horizontal_dist = np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)
            vertical_dist = abs(p1[2] - p2[2])
            
            if horizontal_dist <= horizontal_threshold and vertical_dist <= vertical_threshold:
                valid_edge_count += 1
        
        # If all edges are valid, keep this simplex
        if valid_edge_count == 6:
            valid_simplices.append(simplex)
    
    # Create separate meshes based on connectivity
    mesh_groups = []
    remaining_simplices = set(map(tuple, valid_simplices))
    
    while remaining_simplices:
        current_group = set()
        simplex_queue = [remaining_simplices.pop()]
        
        while simplex_queue:
            current_simplex = simplex_queue.pop()
            current_group.add(current_simplex)
            
            for simplex in list(remaining_simplices):
                if len(set(current_simplex) & set(simplex)) > 0:
                    simplex_queue.append(simplex)
                    remaining_simplices.remove(simplex)
        
        mesh_groups.append(list(current_group))
    
    # Convert mesh groups to PyVista PolyData objects
    pv_meshes = []
    for group in mesh_groups:
        # Format faces for PyVista
        faces = []
        for simplex in group:
            faces.extend([3, simplex[0], simplex[1], simplex[2]])
            faces.extend([3, simplex[0], simplex[1], simplex[3]])
            faces.extend([3, simplex[0], simplex[2], simplex[3]])
            faces.extend([3, simplex[1], simplex[2], simplex[3]])
        
        faces = np.array(faces)
        mesh = pv.PolyData(points, faces)
        pv_meshes.append(mesh)
    
    return pv_meshes

In [56]:
nav_mesh = create_delaunay_meshes(filtered_points, 0.3, 0.4)

In [59]:
visualize_geometry({"red-solid":nav_mesh})

Widget(value='<iframe src="http://localhost:56387/index.html?ui=P_0x3c33b70e0_19&reconnect=auto" class="pyvist…

In [ ]:
na